In [1]:
from urllib.request import urlretrieve
from pyopenms import *
gh = "https://raw.githubusercontent.com/OpenMS/pyopenms-extra/master"
urlretrieve (gh +"/src/data/SimpleSearchEngine_1.mzML", "searchfile.mzML")
urlretrieve (gh +"/src/data/SimpleSearchEngine_1.fasta", "search.fasta")
protein_ids=[]
peptide_ids=[]
SimpleSearchEngineAlgorithm().search("searchfile.mzML", "search.fasta", protein_ids ,peptide_ids)

In [3]:
for peptide_id in peptide_ids:
    print(35*"=")
    print("peptide ID m/z:" , peptide_id.getMZ())
    print("peptide ID rt:" , peptide_id.getRT())
    print("peptide scan index" ,peptide_id.getMetaValue("scan_index"))
    print("peptide scan name" ,peptide_id.getMetaValue("scan_index"))
    print("peptide ID score type:" ,peptide_id.getScoreType())
    
for hit in peptide_id.getHits():
    print("- peptide hit rank:" , hit.getRank())
    print("- peptide hit charge:" , hit.getCharge())
    print("- Peptide hit sequence:", hit.getSequence())
    mz = hit.getSequence().getMonoWeight(Residue.ResidueType.Full, hit.getCharge()) / hit.getCharge()
    print(" - Peptide hit monoisotopic m/z:", mz)
    print(" - Peptide ppm error:", abs(mz - peptide_id.getMZ())/mz *10**6 )
    print(" - Peptide hit score:", hit.getScore())


peptide ID m/z: 520.262817382812
peptide ID rt: 2655.095703125
peptide scan index 0
peptide scan name 0
peptide ID score type: hyperscore
peptide ID m/z: 1063.20983886719
peptide ID rt: 4587.6689453125
peptide scan index 1
peptide scan name 1
peptide ID score type: hyperscore
peptide ID m/z: 775.38720703125
peptide ID rt: 4923.77734375
peptide scan index 2
peptide scan name 2
peptide ID score type: hyperscore
- peptide hit rank: 1
- peptide hit charge: 3
- Peptide hit sequence: RPGADSDIGGFGGLFDLAQAGFR
 - Peptide hit monoisotopic m/z: 775.3854614376374
 - Peptide ppm error: 2.2512591470542094
 - Peptide hit score: 34.93666076660156


In [11]:
tsg= TheoreticalSpectrumGenerator()
thspec = MSSpectrum()
p= Param()
p.setValue("add_metainfo", "true")
tsg.setParameters(p)
peptide=AASequence.fromString("RPGADSDIGGFGGLFDLAQAGFR")
tsg.getSpectrum(thspec,peptide,1,1)

for ion, peak in zip(thspec.getStringDataArrays()[0], thspec):
    print(ion , peak.getMZ())
    
e = MSExperiment()
MzMLFile().load("searchfile.mzML" ,e)
print("spectrum native id", e[2].getNativeID())
mz,i=e[2].get_peaks()
peaks = [(mz,i) for mz,i in zip(mz,i) if i > 1500 and mz > 300]
for peak in peaks:
  print (peak[0], "mz", peak[1], "int")

b'y1+' 175.118952913371
b'b2+' 254.161152072871
b'b3+' 311.18261616857103
b'y2+' 322.187367200471
b'y3+' 379.208831296171
b'b4+' 382.219730328071
b'y4+' 450.245945455671
b'b5+' 497.246674487571
b'y5+' 578.304523710871
b'b6+' 584.278703647071
b'y6+' 649.341637870371
b'b7+' 699.305647806571
b'y7+' 762.425702221271
b'b8+' 812.389712157471
b'b9+' 869.411176253171
b'y8+' 877.452646380771
b'b10+' 926.432640348871
b'y9+' 1024.5210606678709
b'b11+' 1073.501054635971
b'b12+' 1130.522518731671
b'y10+' 1137.6051250187709
b'b13+' 1187.5439828273709
b'y11+' 1194.6265891144708
b'y12+' 1251.6480532101707
b'b14+' 1300.628047178271
b'y13+' 1398.7164674972707
b'b15+' 1447.696461465371
b'y14+' 1455.7379315929707
b'y15+' 1512.7593956886706
b'b16+' 1562.723405624871
b'y16+' 1625.8434600395706
b'b17+' 1675.807469975771
b'y17+' 1740.8704041990707
b'b18+' 1746.844584135271
b'y18+' 1827.9024333585708
b'b19+' 1874.903162390471
b'y19+' 1942.9293775180709
b'b20+' 1945.9402765499708
b'b21+' 2002.9617406456707
b'y2

In [17]:
salgo=SimpleSearchEngineAlgorithm()
p=salgo.getDefaults()
print(p.items())
p[b'precursor:mass_tolerance']=4.0
salgo.setParameters(p)

protein_ids=[]
peptide_ids=[]
salgo.search("searchfile.mzML", "search.fasta", protein_ids, peptide_ids)
print("Found" , len(peptide_ids), "peptides")

[(b'enzyme', 'Trypsin'), (b'decoys', 'false'), (b'precursor:mass_tolerance', 10.0), (b'precursor:mass_tolerance_unit', 'ppm'), (b'precursor:min_charge', 2), (b'precursor:max_charge', 5), (b'precursor:isotopes', [0, 1]), (b'fragment:mass_tolerance', 10.0), (b'fragment:mass_tolerance_unit', 'ppm'), (b'modifications:fixed', [b'Carbamidomethyl (C)']), (b'modifications:variable', [b'Oxidation (M)']), (b'modifications:variable_max_per_peptide', 2), (b'annotate:PSM', [b'ALL']), (b'peptide:min_size', 7), (b'peptide:max_size', 40), (b'peptide:missed_cleavages', 1), (b'peptide:motif', ''), (b'report:top_hits', 1)]
Found 3 peptides


In [18]:
from urllib.request import urlretrieve
searchfile = "../../src/data/BSA1.mzML"
searchdb = "../../src/data/18Protein_SoCe_Tr_detergents_trace.fasta"

targets =list()
decoys =list()
FASTAFile().load(searchdb, targets) 
decoy_generator = DecoyGenerator()
for entry in targets:
    rev_entry = FASTAEntry(entry) 
    rev_entry.identifier = "DECOY_" + rev_entry.identifier 
    aas = AASequence().fromString(rev_entry.sequence) 
    rev_entry.sequence = decoy_generator.reverseProtein(aas).toString() # reverse
    decoys.append(rev_entry)

target_decoy_database = "search_td.fasta"
FASTAFile().store(target_decoy_database, targets + decoys) 
protein_ids =[]
peptide_ids =[]

simplesearch = SimpleSearchEngineAlgorithm()
params = simplesearch.getDefaults()
score_annot = [b'fragment_mz_error_median_ppm', b'precursor_mz_error_ppm']
params.setValue(b'annotate:PSM', score_annot)
params.setValue(b'peptide:max_size', 30)
simplesearch.setParameters(params)

simplesearch.search(searchfile, target_decoy_database, protein_ids, peptide_ids)

FalseDiscoveryRate().apply(peptide_ids)

idfilter = IDFilter()
idfilter.filterHitsByScore(peptide_ids, 0.01)
idfilter.removeDecoyHits(peptide_ids)

IdXMLFile().store("searchfile_results_1perc_FDR.idXML", protein_ids, peptide_ids)

RuntimeError: the file '../../src/data/18Protein_SoCe_Tr_detergents_trace.fasta' could not be found

In [20]:
protein_ids = []
peptide_ids = []

# Re-run search since we need to keep decoy hits for inference
simplesearch.search(searchfile, target_decoy_database, protein_ids, peptide_ids)

# Run inference
BasicProteinInferenceAlgorithm().run(peptide_ids, protein_ids)

# Annotate q-value on protein level
FalseDiscoveryRate().apply(protein_ids)

# Filter by 1% protein FDR (q-value < 0.01)
idfilter = IDFilter()
idfilter.filterHitsByScore(protein_ids, 0.01)
idfilter.removeDecoyHits(protein_ids)
# Restore valid references into the proteins
remove_peptides_without_reference = True
idfilter.updateProteinReferences(peptide_ids, protein_ids, remove_peptides_without_reference)

# store protein-FDR filtered
IdXMLFile().store("searchfile_results_1perc_protFDR.idXML", protein_ids, peptide_ids)

NameError: name 'SimpleSearch' is not defined